# 先抓出news裡面對應是market 2天后大漲或大跌的那些文章

## 先把market 2天后大漲或大跌的那些日期抓出來

In [1]:
import csv 
from datetime import datetime
from datetime import timedelta

file_market = open('market_ready.csv', 'r')
reader_market = csv.DictReader(file_market)

time_delta = timedelta(days = -1)

up_list = []
up_intervel = []
down_list = []
down_intervel = []
for a_row in reader_market:
    two_day_return = float(a_row[u'2日報酬'])
    
    if two_day_return <= -0.0124:
        date = datetime.strptime( a_row['date'], "%Y-%m-%d")
        down_list.append( date.date() )
        down_intervel.append( (date + time_delta).date() )
        down_intervel.append( (date + time_delta + time_delta).date() )
    elif two_day_return >= 0.0135:
        date = datetime.strptime( a_row['date'], "%Y-%m-%d") 
        up_list.append(date.date())
        up_intervel.append( (date + time_delta).date() )
        up_intervel.append( (date + time_delta + time_delta).date() )
   
                 



In [2]:
len(up_list), len(down_list)

(74, 73)

## 有三天是有重疊的... 大跌之後跟著大漲 或著相反

In [3]:
set(up_intervel) & set(down_intervel)

{datetime.date(2016, 11, 7),
 datetime.date(2018, 3, 25),
 datetime.date(2018, 12, 3)}

In [4]:
time_delta = timedelta(days = 1) #可看出到底是哪一天是漲 哪一天是跌
intersect = set(up_intervel) & set(down_intervel)
for i in intersect:
    print( '{0} in up_list'.format(i+time_delta ), (i+time_delta) in up_list )
    print( '{0} in down_list'.format(i+time_delta ), (i+time_delta) in down_list )

2018-03-26 in up_list False
2018-03-26 in down_list True
2016-11-08 in up_list True
2016-11-08 in down_list False
2018-12-04 in up_list True
2018-12-04 in down_list False


## 所以我們知道 11/8是漲的樣本，11/9是跌的樣本，以此類推
## 下面我們可以得知，11/8是不需要留下的，只需要留下11/9的樣本即可

In [5]:
a_list = []
time_delta = timedelta(days = 1)
for i in intersect:
    a_list.append( i )
    a_list.append( i+time_delta )
    a_list.append( i+time_delta +time_delta)
a_list

[datetime.date(2018, 3, 25),
 datetime.date(2018, 3, 26),
 datetime.date(2018, 3, 27),
 datetime.date(2016, 11, 7),
 datetime.date(2016, 11, 8),
 datetime.date(2016, 11, 9),
 datetime.date(2018, 12, 3),
 datetime.date(2018, 12, 4),
 datetime.date(2018, 12, 5)]

In [6]:
import csv 
from datetime import datetime
from datetime import timedelta

file_market = open('market_ready.csv', 'r')
reader_market = csv.DictReader(file_market)



for a_row in reader_market:
    price = float(a_row['price'])
    date = datetime.strptime( a_row['date'], "%Y-%m-%d").date()
    one_day_ret = float(a_row[u'1日報酬'])
    two_day_ret = float(a_row[u'2日報酬'])
    if date in a_list:
        print(date, price, one_day_ret,two_day_ret)

2016-11-07 9189.84 0.013419495707503792 0.013517850466568185
2016-11-08 9217.43 0.0030022285480487305 0.016462012648666008
2016-11-09 8943.2 -0.029751243025441968 -0.026838334508544153
2018-03-26 10840.05 0.0015448110701604169 -0.015063820662484724
2018-03-27 10986.79 0.013536837929714494 0.015102560856963702
2018-12-03 10137.87 0.02526691363193681 0.025543834518924977
2018-12-04 10083.54 -0.005359113896706106 0.01977239146725892
2018-12-05 9916.74 -0.01654180972158598 -0.021812274175936463


## 只保留重疊的後一天

In [10]:
to_del = []
time_delta = timedelta(days = 1)
for i in intersect:
    to_del.append( i+time_delta )
    

file_market = open('market_ready.csv', 'r')
reader_market = csv.DictReader(file_market)

time_delta = timedelta(days = -1)

up_list = []
up_intervel = []
down_list = []
down_intervel = []
for a_row in reader_market:
    two_day_return = float(a_row[u'2日報酬'])
    date = datetime.strptime( a_row['date'], "%Y-%m-%d")
    if two_day_return <= -0.0124 and date.date() not in to_del:
        
        down_list.append( date )
        down_intervel.append( (date + time_delta).date() )
        down_intervel.append( (date + time_delta + time_delta).date() )
    elif two_day_return >= 0.0135 and date.date() not in to_del:
        
        up_list.append(date)
        up_intervel.append( (date + time_delta).date() )
        up_intervel.append( (date + time_delta + time_delta).date() )
  
                 



In [11]:
len(up_list), len(down_list), set(up_intervel) & set(down_intervel)


(72, 72, set())

# 以下開始為新聞的EDA，初步先把news裡面的垃圾訊息排除，並留下有用新聞

In [70]:
df111 = pd.read_csv(r'C:\Users\User\Desktop\大數據與商業分析\mid tern\news.csv', encoding = 'big5')
df111

,id,p_type,s_name,s_area_name,post_time,title,author,content,page_url
0,1451580107540_N01,news,yahoo股市,重大要聞,2016/1/1 00:06:00,淪落的2015 竟是78年來最難獲利的一年,鉅亨網,鉅亨網編譯郭照青這一年，獲利真是難上難。<BR>根據CNBC分析報導，興業銀行的數據顯示，2...,https://tw.stock.yahoo.com/news_content/url/d/...
1,1451582807314_N01,news,yahoo股市,重大要聞,2016/1/1 00:52:00,美股盤中─今年最後交易日 道瓊大跌逾150點 注意油價走勢,鉅亨網,鉅亨網編譯郭照青今日為2015年最後交易日，美國股市中場走低，投資人注意油價走勢。<BR>道...,https://tw.stock.yahoo.com/news_content/url/d/...
2,1451584607515_N01,news,yahoo股市,重大要聞,2016/1/1 01:21:00,2016年 原油將突破每桶80美元,鉅亨網,鉅亨網編譯郭照青根據路透社分析師Andy Critchlow指出，原油價格下一波的大幅走勢將...,https://tw.stock.yahoo.com/news_content/url/d/...
3,1451597207172_N01,news,yahoo股市,重大要聞,2016/1/1 02:44:00,2015年 美國近70%投資人賠錢,鉅亨網,鉅亨網編譯郭照青拜拜了，2015年。<BR>根據CNNMoney分析報導，應用程式Openf...,https://tw.stock.yahoo.com/news_content/url/d/...
4,1451599007463_N01,news,yahoo股市,重大要聞,2016/1/1 03:30:00,【歐股盤後】全面收黑,中央社,（中央社台北2016年1月1日電）歐洲股市31日封關，三大股市全都收黑，倫敦股市提早收盤，金...,https://tw.stock.yahoo.com/news_content/url/d/...
5,1451599907245_N01,news,yahoo股市,重大要聞,2016/1/1 04:27:00,貴金屬盤後─黃金收高0.40美元 但今年大跌10.5%,鉅亨網,鉅亨網編譯郭照青2015年最後交易日，黃金期貨週四小幅上揚，但本週，本月與今年均下跌。<BR...,https://tw.stock.yahoo.com/news_content/url/d/...
6,1451600807160_N01,news,yahoo股市,重大要聞,2016/1/1 05:30:00,美股盤後─調整投資組合賣壓 道瓊收低178點 成交量清淡,鉅亨網,鉅亨網編譯郭照青週四，2015年最後交易日，儘管油價回穩，美國股市仍大幅收低，成交量清淡，<...,https://tw.stock.yahoo.com/news_content/url/d/...
7,1451603507368_N01,news,yahoo股市,重大要聞,2016/1/1 06:20:00,歐股全面收黑,法新社,（法新社倫敦31日電） 歐洲股市今天封關，三大股市全都收黑，倫敦股市提早收盤，金融時報百種股...,https://tw.stock.yahoo.com/news_content/url/d/...
8,1451603508604_N01,news,yahoo股市,重大要聞,2016/1/1 06:29:00,【美股盤後】封關收黑,中央社,（中央社台北2016年1月1日電）華爾街股市31日封關，揮別2015年，三大指數全都收黑，道...,https://tw.stock.yahoo.com/news_content/url/d/...
9,1451604407512_N01,news,yahoo股市,重大要聞,2016/1/1 06:57:00,【能源盤後】全年重挫三成,中央社,（中央社台北2016年1月1日電）世界油價31日上揚，年末雖跌幅縮減，但2015年仍以大跌作...,https://tw.stock.yahoo.com/news_content/url/d/...


In [71]:
df111.describe()

,id,p_type,s_name,s_area_name,post_time,title,author,content,page_url
count,247273,247273,247273,247273,247273,247273,247258,237798,247273
unique,247273,1,1,2,143138,236819,11,234135,247273
top,1531092438005_N01,news,yahoo股市,最新財經新聞,2018/6/28 08:29:00,工商時報晨訊,時報資訊,同上,https://tw.stock.yahoo.com/news_content/url/d/...
freq,1,247273,247273,139215,41,489,76713,2237,1


In [72]:
df112 = df111[df111['content']!=u'同上']

In [73]:
df112.describe()

,id,p_type,s_name,s_area_name,post_time,title,author,content,page_url
count,245036,245036,245036,245036,245036,245036,245021,235561,245036
unique,245036,1,1,2,142845,234617,11,234134,245036
top,1531092438005_N01,news,yahoo股市,最新財經新聞,2018/6/27 08:05:00,工商時報晨訊,時報資訊,待續,https://tw.stock.yahoo.com/news_content/url/d/...
freq,1,245036,245036,136978,41,489,76713,270,1


In [74]:
df113 = df112[df112['content']!=u'待續']
df113.describe()

,id,p_type,s_name,s_area_name,post_time,title,author,content,page_url
count,244766,244766,244766,244766,244766,244766,244751,235291,244766
unique,244766,1,1,2,142845,234614,11,234133,244766
top,1531092438005_N01,news,yahoo股市,最新財經新聞,2018/6/27 08:05:00,工商時報晨訊,時報資訊,本日資訊傳送完畢,https://tw.stock.yahoo.com/news_content/url/d/...
freq,1,244766,244766,136711,41,489,76713,101,1


In [75]:
df114 = df113[df113['content']!=u'本日資訊傳送完畢']
df114.describe()

,id,p_type,s_name,s_area_name,post_time,title,author,content,page_url
count,244665,244665,244665,244665,244665,244665,244650,235190,244665
unique,244665,1,1,2,142812,234613,11,234132,244665
top,1531092438005_N01,news,yahoo股市,最新財經新聞,2018/6/28 08:29:00,工商時報晨訊,時報資訊,如何在Yahoo股市APP看直播？<BR> ★IOS用戶：請點選右上角的「新聞」，再點擊上方...,https://tw.stock.yahoo.com/news_content/url/d/...
freq,1,244665,244665,136610,41,489,76713,22,1


In [95]:
mask = df114['content'].str.contains(u'如何在Yahoo股市APP看直播', regex=False, na = False)
df115 = df114[~mask]
df115.describe()

,id,p_type,s_name,s_area_name,post_time,title,author,content,page_url
count,244630,244630,244630,244630,244630,244630,244615,235155,244630
unique,244630,1,1,2,142780,234578,11,234127,244630
top,1531092438005_N01,news,yahoo股市,最新財經新聞,2018/6/28 08:29:00,工商時報晨訊,時報資訊,近期上市上櫃公司實施庫藏股一覽表：<BR>代號 公司名 買回張數 區間價格(元) 買回期間<...,https://tw.stock.yahoo.com/news_content/url/d/...
freq,1,244630,244630,136610,41,489,76713,7,1


In [96]:
mask = df115['content'].str.contains(u'獎號碼', regex=False, na = False)
df116 = df115[~mask]

In [97]:
mask = df116['content'].str.contains(u'威力彩', regex=False, na = False)
df117 = df116[~mask]

In [99]:
mask = df117['content'].str.contains(u'彩券', regex=False, na = False)
df118 = df117[~mask]

In [106]:
mask = df118['title'].str.contains(u'統一發票', regex=False, na = False)
df119 = df118[~mask]

In [110]:
mask = df119['title'].str.contains(u'頭獎', regex=False, na = False)
df120 = df119[~mask]

In [112]:
mask = df120['title'].str.contains(u'行事曆', regex=False, na = False)
df121 = df120[~mask]
df120[mask]

,id,p_type,s_name,s_area_name,post_time,title,author,content,page_url
614,1452211676184_N01,news,yahoo股市,重大要聞,2016/1/8 08:00:00,2016/1/8財經行事曆,Yahoo奇摩股市,2016/1/8財經行事曆<BR>今日國內外財經焦點：<BR>台灣：<BR>1.新股掛牌：F...,https://tw.stock.yahoo.com/news_content/url/d/...
2322,1453768129515_N01,news,yahoo股市,重大要聞,2016/1/26 08:15:00,1/26財經行事曆,Yahoo奇摩股市,今日國內外財經焦點：<BR>台灣：<BR>1.新股掛牌：宏觀(6568)<BR>2.股東會：...,https://tw.stock.yahoo.com/news_content/url/d/...
2410,1453805918029_N01,news,yahoo股市,重大要聞,2016/1/26 18:45:00,2016/1/27財經行事曆,Yahoo奇摩股市,今日國內外財經焦點：<BR>台灣：<BR>1.新股掛牌：F-英利(2239)、F-紅馬(29...,https://tw.stock.yahoo.com/news_content/url/d/...
2568,1453892318745_N01,news,yahoo股市,重大要聞,2016/1/27 18:40:00,2016/1/28財經行事曆,Yahoo奇摩股市,今日國內外財經焦點：<BR>台灣：<BR>1.股東會：矽品(2325)、佳晶科(5242)、...,https://tw.stock.yahoo.com/news_content/url/d/...
2711,1453978130035_N01,news,yahoo股市,重大要聞,2016/1/28 18:31:00,2016/1/29財經行事曆,Yahoo奇摩股市,今日國內外財經焦點：<BR>台灣：<BR>1.新股掛牌：心悅(6575)<BR>2.股東會：...,https://tw.stock.yahoo.com/news_content/url/d/...
3023,1454228329825_N01,news,yahoo股市,重大要聞,2016/1/31 16:10:00,2016/2/1財經行事曆,Yahoo奇摩股市,今日國內外財經焦點：<BR>台灣：<BR>1.法說會：聯發科(2454)<BR>中國：<BR...,https://tw.stock.yahoo.com/news_content/url/d/...
3111,1454321034477_N01,news,yahoo股市,重大要聞,2016/2/1 17:47:00,2016/2/2財經行事曆,Yahoo奇摩股市,今日國內外財經焦點：<BR>台灣：<BR>1.法說會：超豐(2441)、聯詠(3034)、文...,https://tw.stock.yahoo.com/news_content/url/d/...
3244,1454409228261_N01,news,yahoo股市,重大要聞,2016/2/2 18:16:00,2016/2/3財經行事曆,Yahoo奇摩股市,今日國內外財經焦點：<BR>台灣：<BR>1.股東會：千興(2025)<BR>2.法說會：宏...,https://tw.stock.yahoo.com/news_content/url/d/...
3376,1454504173153_N01,news,yahoo股市,重大要聞,2016/2/3 20:39:00,2016/2/4財經行事曆,Yahoo奇摩股市,今日國內外財經焦點：<BR>台灣：<BR>1.台股今日休市<BR>2.股東會：聯亞藥(656...,https://tw.stock.yahoo.com/news_content/url/d/...
3448,1454581573947_N01,news,yahoo股市,重大要聞,2016/2/4 18:15:00,2016/2/5財經行事曆,Yahoo奇摩股市,今日國內外財經焦點：<BR>台灣：<BR>1.台股今日休市<BR>2.股東會：華廣(4737...,https://tw.stock.yahoo.com/news_content/url/d/...


In [118]:
df_len = df121['content'].fillna('').apply(len)
df_len.describe()

count    243041.000000
mean        607.546513
std         498.083274
min           0.000000
25%         271.000000
50%         526.000000
75%         817.000000
max       16338.000000
Name: content, dtype: float64

In [123]:
mask = df121[df_len <=30 ]
mask

,id,p_type,s_name,s_area_name,post_time,title,author,content,page_url,length_content
107,1451867212458_N01,news,yahoo股市,重大要聞,2016/1/4 07:58:00,廢證所稅 催化資金動能(工商時報，無內文),Moneydj理財網,NaN,https://tw.stock.yahoo.com/news_content/url/d/...,243041
108,1451867213978_N01,news,yahoo股市,重大要聞,2016/1/4 07:58:00,退撫+勞動新銀彈 3240億元入台股(工商時報，無內文),Moneydj理財網,NaN,https://tw.stock.yahoo.com/news_content/url/d/...,243041
109,1451867215197_N01,news,yahoo股市,重大要聞,2016/1/4 07:59:00,住宅去年使照核發戶數 7年新高(工商時報，無內文),Moneydj理財網,NaN,https://tw.stock.yahoo.com/news_content/url/d/...,243041
110,1451867216394_N01,news,yahoo股市,重大要聞,2016/1/4 07:59:00,淡季效應 蘋果供應鏈盤整(工商時報，無內文),Moneydj理財網,NaN,https://tw.stock.yahoo.com/news_content/url/d/...,243041
111,1451867217656_N01,news,yahoo股市,重大要聞,2016/1/4 08:00:00,售屋所得課稅率 南增北降(經濟日報，無內文),Moneydj理財網,NaN,https://tw.stock.yahoo.com/news_content/url/d/...,243041
480,1452123132697_N01,news,yahoo股市,重大要聞,2016/1/7 07:09:00,今日記事,財訊快報,1.大中(6435)今日掛牌上櫃交易。,https://tw.stock.yahoo.com/news_content/url/d/...,243041
488,1452126735129_N01,news,yahoo股市,重大要聞,2016/1/7 08:02:00,戴德梁行：今年房市 8年寒冬開始(工商時報，無內文),Moneydj理財網,NaN,https://tw.stock.yahoo.com/news_content/url/d/...,243041
489,1452126736971_N01,news,yahoo股市,重大要聞,2016/1/7 08:03:00,北韓氫彈試爆 東北亞緊張(經濟日報，無內文),Moneydj理財網,NaN,https://tw.stock.yahoo.com/news_content/url/d/...,243041
616,1452212568928_N01,news,yahoo股市,重大要聞,2016/1/8 08:03:00,出口恐連11黑 七年最遭(經濟日報，無內文),Moneydj理財網,NaN,https://tw.stock.yahoo.com/news_content/url/d/...,243041
617,1452212570150_N01,news,yahoo股市,重大要聞,2016/1/8 08:03:00,財部救台股 急call公股護盤(經濟日報，無內文),Moneydj理財網,NaN,https://tw.stock.yahoo.com/news_content/url/d/...,243041


In [130]:
df122 = df121[ df_len > 30 ].drop('length_content', axis = 1)
df122

,id,p_type,s_name,s_area_name,post_time,title,author,content,page_url
0,1451580107540_N01,news,yahoo股市,重大要聞,2016/1/1 00:06:00,淪落的2015 竟是78年來最難獲利的一年,鉅亨網,鉅亨網編譯郭照青這一年，獲利真是難上難。<BR>根據CNBC分析報導，興業銀行的數據顯示，2...,https://tw.stock.yahoo.com/news_content/url/d/...
1,1451582807314_N01,news,yahoo股市,重大要聞,2016/1/1 00:52:00,美股盤中─今年最後交易日 道瓊大跌逾150點 注意油價走勢,鉅亨網,鉅亨網編譯郭照青今日為2015年最後交易日，美國股市中場走低，投資人注意油價走勢。<BR>道...,https://tw.stock.yahoo.com/news_content/url/d/...
2,1451584607515_N01,news,yahoo股市,重大要聞,2016/1/1 01:21:00,2016年 原油將突破每桶80美元,鉅亨網,鉅亨網編譯郭照青根據路透社分析師Andy Critchlow指出，原油價格下一波的大幅走勢將...,https://tw.stock.yahoo.com/news_content/url/d/...
3,1451597207172_N01,news,yahoo股市,重大要聞,2016/1/1 02:44:00,2015年 美國近70%投資人賠錢,鉅亨網,鉅亨網編譯郭照青拜拜了，2015年。<BR>根據CNNMoney分析報導，應用程式Openf...,https://tw.stock.yahoo.com/news_content/url/d/...
4,1451599007463_N01,news,yahoo股市,重大要聞,2016/1/1 03:30:00,【歐股盤後】全面收黑,中央社,（中央社台北2016年1月1日電）歐洲股市31日封關，三大股市全都收黑，倫敦股市提早收盤，金...,https://tw.stock.yahoo.com/news_content/url/d/...
5,1451599907245_N01,news,yahoo股市,重大要聞,2016/1/1 04:27:00,貴金屬盤後─黃金收高0.40美元 但今年大跌10.5%,鉅亨網,鉅亨網編譯郭照青2015年最後交易日，黃金期貨週四小幅上揚，但本週，本月與今年均下跌。<BR...,https://tw.stock.yahoo.com/news_content/url/d/...
6,1451600807160_N01,news,yahoo股市,重大要聞,2016/1/1 05:30:00,美股盤後─調整投資組合賣壓 道瓊收低178點 成交量清淡,鉅亨網,鉅亨網編譯郭照青週四，2015年最後交易日，儘管油價回穩，美國股市仍大幅收低，成交量清淡，<...,https://tw.stock.yahoo.com/news_content/url/d/...
7,1451603507368_N01,news,yahoo股市,重大要聞,2016/1/1 06:20:00,歐股全面收黑,法新社,（法新社倫敦31日電） 歐洲股市今天封關，三大股市全都收黑，倫敦股市提早收盤，金融時報百種股...,https://tw.stock.yahoo.com/news_content/url/d/...
8,1451603508604_N01,news,yahoo股市,重大要聞,2016/1/1 06:29:00,【美股盤後】封關收黑,中央社,（中央社台北2016年1月1日電）華爾街股市31日封關，揮別2015年，三大指數全都收黑，道...,https://tw.stock.yahoo.com/news_content/url/d/...
9,1451604407512_N01,news,yahoo股市,重大要聞,2016/1/1 06:57:00,【能源盤後】全年重挫三成,中央社,（中央社台北2016年1月1日電）世界油價31日上揚，年末雖跌幅縮減，但2015年仍以大跌作...,https://tw.stock.yahoo.com/news_content/url/d/...


In [131]:
df_final = df122.drop_duplicates(subset ='content')
df_final.describe()

,id,p_type,s_name,s_area_name,post_time,title,author,content,page_url
count,232514,232514,232514,232514,232514,232514,232499,232514,232514
unique,232514,1,1,2,135037,222873,11,232514,232514
top,1531092438005_N01,news,yahoo股市,最新財經新聞,2018/6/28 08:29:00,工商時報晨訊,時報資訊,（中央社記者鄭崇生華盛頓2017年4月2日專電）美國總統川普今天說，他非常尊重中國國家主席習...,https://tw.stock.yahoo.com/news_content/url/d/...
freq,1,232514,232514,135083,40,488,68079,1,1


In [132]:
df_final.to_csv('news_ready.csv', encoding = 'big5')

## 經過以上步驟，我將重複的、關於彩票及發票的、關於行事曆、以及內容過少的新聞排除

# 接著讀取文章，並分類為上漲及下跌文章群

In [12]:
for date in up_list:
        up_lower = date + timedelta(days = -1, hours = -10) #1/31號的大漲，我們要抓的新聞期間下限是1/29 14:00開始(因為會有關盤資訊)
        up_upper = date + timedelta( hours = -15) #新聞期間下限是1/30 9:00 結束
        print(date, up_lower,up_upper)

2016-01-25 00:00:00 2016-01-23 14:00:00 2016-01-24 09:00:00
2016-01-29 00:00:00 2016-01-27 14:00:00 2016-01-28 09:00:00
2016-01-30 00:00:00 2016-01-28 14:00:00 2016-01-29 09:00:00
2016-02-16 00:00:00 2016-02-14 14:00:00 2016-02-15 09:00:00
2016-02-17 00:00:00 2016-02-15 14:00:00 2016-02-16 09:00:00
2016-02-26 00:00:00 2016-02-24 14:00:00 2016-02-25 09:00:00
2016-03-01 00:00:00 2016-02-28 14:00:00 2016-02-29 09:00:00
2016-03-02 00:00:00 2016-02-29 14:00:00 2016-03-01 09:00:00
2016-03-03 00:00:00 2016-03-01 14:00:00 2016-03-02 09:00:00
2016-03-17 00:00:00 2016-03-15 14:00:00 2016-03-16 09:00:00
2016-03-31 00:00:00 2016-03-29 14:00:00 2016-03-30 09:00:00
2016-04-14 00:00:00 2016-04-12 14:00:00 2016-04-13 09:00:00
2016-05-23 00:00:00 2016-05-21 14:00:00 2016-05-22 09:00:00
2016-05-24 00:00:00 2016-05-22 14:00:00 2016-05-23 09:00:00
2016-05-30 00:00:00 2016-05-28 14:00:00 2016-05-29 09:00:00
2016-06-08 00:00:00 2016-06-06 14:00:00 2016-06-07 09:00:00
2016-06-20 00:00:00 2016-06-18 14:00:00 

In [14]:
import csv 
from datetime import datetime
from datetime import timedelta

news_content_plus_title = dict()
news_content_plus_title['up'] = [] 
news_content_plus_title['down'] = [] 


with open( 'news_ready.csv', encoding = 'big5') as f:

    csv_reader = csv.DictReader(f)
    for a_row in csv_reader:
        time = datetime.strptime( a_row['post_time'], "%Y/%m/%d %H:%M:%S") 
        title = a_row['title']
        content = a_row['content']
        for date in up_list:
            up_lower = date + timedelta(days = -1, hours = -10) #1/31號的大漲，我們要抓的新聞期間下限是1/29 14:00開始(因為會有關盤資訊)
            up_upper = date + timedelta( hours = -15) #新聞期間下限是1/30 9:00 結束
            if up_lower < time < up_upper:
                news_content_plus_title['up'].append( title +'＜' +content   )
                break

        for date in down_list:           
            down_lower = date + timedelta(days = -1, hours = -10)
            down_upper = date + timedelta( hours = -15) 
            if down_lower< time < down_upper:
                news_content_plus_title['down'].append( title +'＜' +content   )
                break


In [15]:
import pandas as pd
df = pd.DataFrame.from_dict(news_content_plus_title, 'index')
df

,0,1,2,3,4,5,6,7,8,9,...,12844,12845,12846,12847,12848,12849,12850,12851,12852,12853
up,《政治》獲頒當選證書，蔡英文：人民給的＜中選會昨頒贈當選證書給正副總統當選人蔡英文及陳建仁。...,BBC妙聞 iPhone用谷歌搜尋 蘋果收300億＜（中央社倫敦2016年1月22日綜合外電...,《政治》在美受訓墜毀，我F-16飛官罹難＜中華民國空軍飛行員高鼎程赴美路克基地受訓，台灣時間...,社福津貼調高囉 每月增加73到600元＜（中央社記者陳政偉台北2016年1月23日電）國家發...,技嘉葉培城：今年網通、電競產品帶頭 獲利拚回2014年水位＜鉅亨網記者蔡宗憲 台北主機板大廠...,技嘉布局新領域產品 從手持裝置往外擴散＜鉅亨網記者蔡宗憲 台北主機板廠技嘉 (2376) 今...,主辦奧運日近 巴西經濟危機罩頂＜（法新社巴西聖保羅23日電） 巴西期待2016年有好的開始，...,迎接奧運年 巴西經濟危機罩頂＜（法新社巴西聖保羅23日電） 巴西期待2016年有好的開始，今...,世界主要報紙頭條新聞標題之二＜（中央社台北2016年1月23日綜合報導）以下是世界各地主要報...,法經長質疑 大陸官方誇大成長數字＜（法新社達佛斯22日電） 法國經濟部長馬卡龍（Emmanu...,...,None,None,None,None,None,None,None,None,None,None
down,《國際金融》全球股市最慘開年，首周跌逾2.3兆美元＜受中國經濟亂流攪局，全球股市開年以來出師...,《政治》民進黨宣告：未來對外經貿談判將採黑箱作業＜民進黨表示，未來兩岸及國際經貿談判應仿效美...,《電腦設備》英業達法人積極買進＜NB代工廠英業達 （2356） 上周股價相對持穩，單周僅下跌...,花花公子創辦人海夫納傳將出售豪宅＜（法新社洛杉磯10日電） 美國媒體報導，「花花公子」（Pl...,美元兌日圓做多者一夜白了頭、週線創兩年最大貶幅＜MoneyDJ新聞 2016-01-11 0...,《美股》道瓊五日崩逾千點、史上最慘開春！VIX飆＜MoneyDJ新聞 2016-01-11 ...,《美債》就業人數增、薪情卻欠佳！殖利率逾10週低＜MoneyDJ新聞 2016-01-11 ...,日股今日因國定假日「成人日」休市一天＜MoneyDJ新聞 2016-01-11 07:08:...,晨間解析：蘋概股反攻，台股收復10年線，選前法人觀望，周K下挫444點＜中國股匯重挫、油價大...,今日記事＜1.嘉晶(3016)合併漢磊增資新股權利證書上市。<BR>2.夠麻吉(8472)今...,...,兆勁董事會通過董事候選人提名名單＜公開資訊觀測站重大訊息公告<BR>(2444)兆勁-公告本...,《大陸產業》海航傳售英邁國際，開價75億美元＜債台高築的海航集團昨（24）日再度傳出要出售資...,富裔：董事蔡佩珍擬向公司承購新北市新店區復興路102號16樓房地壹戶，計5800萬元＜公開資...,原物料：需求低迷，LME期鋁週一收低0.8%，盤中觸及17個月低點＜投資人對全球經濟增長放緩...,仍看漲2019年 華爾街：至少還會再漲一年 S&P 500漲15%＜儘管 12 月市場動盪，...,《大陸產業》TCL賤賣資產？深交所緊盯＜大陸知名家電企業、在深圳股市上市的TCL集團，擬以4...,《半導體》旺宏、IBM簽約，續開發相變化記憶體＜記憶體廠旺宏 (2337) 決議與IBM簽訂...,《期貨》1月台指期開盤跌逾百點，金融期及電子期齊開低＜空方大軍壓境，1月台指期開盤跌逾百點，...,《港股》港股首家「新三板＋H」，君實生物上市首日大漲22％＜香港股市第1家「新三板+H」公司...,《大陸經濟》加速充電基礎設施發展，陸電動汽車充電樁近73萬台＜近幾年大陸積極推動電動汽車發展...


In [16]:
df1 = df.T
print(df1.shape)
print(df1['up'].describe())
df1['down'].describe()

(12854, 2)
count                                                  9277
unique                                                 9277
top       《證交所》今日單一證券鉅額成交股票：台泥、美食-KY、兆豐金＜1.元展46 (03049X)...
freq                                                      1
Name: up, dtype: object


count                                                 12854
unique                                                12854
top       營收：光菱(8032)9月營收3億3898萬元，月增率-1.32%，年增率-5.41%＜光菱...
freq                                                      1
Name: down, dtype: object

In [17]:
df1.to_csv('news_UandD.csv', encoding = 'big5')

# 現在我們來建構用來訓練的數據，keywords是由news_UandD篩選而出

In [9]:
import pandas as pd
df = pd.read_csv('keywords.csv', encoding = 'big5')
df.head()

,word,type
0,產金分離,up
1,台股開紅盤,up
2,參加富邦證券,up
3,股開紅盤,up
4,華為事件,up


In [18]:
keyword_list = df['word'].tolist()
keyword_list

['產金分離',
 '台股開紅盤',
 '參加富邦證券',
 '股開紅盤',
 '華為事件',
 '安能得',
 '參加日盛證券',
 '金分離',
 '華為財務長',
 '波索納洛',
 '保釋',
 '台北君悅酒店',
 '為事',
 '孟晚舟',
 '瑪莉亞',
 '蔡明忠',
 '禁售',
 '買回股份期間',
 '際已買回股數',
 '實際已買回股',
 '買回股數占預',
 '股數占預定買',
 '數占預定買回',
 '已買回股數占',
 '回股數占預定',
 '鉅亨網編譯郭',
 '網編譯郭照青',
 '亨網編譯郭照',
 '際買回股份期',
 '占預定買回股',
 '復興航空',
 '實際買回股份',
 '份期間',
 '日盛證券',
 '封關日',
 '預定買回股數',
 '產金',
 '為財',
 '被捕',
 '週一收',
 '米手',
 '股份占',
 '英國公',
 '託證券',
 '公司股價低於',
 '君悅',
 '跌幅約為',
 '條規定之',
 '台北市信義區',
 '資產型',
 '小米手',
 '櫃公司買回本',
 '市上櫃公司買',
 '上櫃公司買回',
 '公司股份辦法',
 '本公司股份辦',
 '區間價格下限',
 '順藥',
 '執行買回',
 '國泰證券',
 '梅克爾',
 '馬克宏',
 '一收',
 '富邦證券',
 '買回股份之',
 '太複合',
 '福州',
 '療法',
 '司買回本公司',
 '公司買回本公',
 '開紅盤',
 '會已考慮公司',
 '事會已考慮公',
 '事會決議買回',
 '董事會決議買',
 '董事會已考慮',
 '已考慮公司財',
 '考慮公司財務',
 '慮公司財務狀',
 '居住證',
 '富邦證',
 '僅占本公司',
 '第十條',
 '口值',
 '標收',
 '腸病毒',
 '退票',
 '裁決',
 '克爾',
 '買回公司股份',
 '已考慮',
 '成長多',
 '婦聯會',
 '補跌',
 '興航',
 '目標收益',
 '盛證券',
 '場高收益債券',
 '自當',
 '動準備',
 '脫歐的',
 '市場高收益債',
 '下指紋',
 '多重資產型',
 '占本公司',
 '產動態',
 '興市場高收益',
 '新興市場高收',
 '五美股',
 '

## 把同日的每個文章batch一起，並整理成完整的資料集

In [1]:
target = dict()
import csv 
from datetime import datetime
from datetime import timedelta

file_market = open('market_ready.csv', 'r')
reader_market = csv.DictReader(file_market)
market_date_list = []


for a_row in reader_market:
    date = datetime.strptime( a_row['date'], "%Y-%m-%d").date()
    two_day_returns = float(a_row[u'2日報酬'])
    market_date_list.append(date)
    if two_day_returns <= 0 : 
        target[date] = 0
    elif two_day_returns > 0 :
        target[date] = 1


### 假設1/3號有開盤，那其對應的新聞區間為1/1 14:00 ~ 1/2 9:00，那麼把新聞日期減去14小時後，如果小時數介於0~19之間，那其對應的收盤日就是新聞日期減去14小時的日期再加上兩天

In [4]:
import csv 
from datetime import datetime
from datetime import timedelta

news_content_plus_title = dict()
number_of_document = dict()
'''
news_content_plus_title[date] = '文章字串'
'''

time_delta = timedelta(hours = -14)

with open( 'news_ready.csv', encoding = 'big5') as f:

    csv_reader = csv.DictReader(f)
    for a_row in csv_reader:
        time = datetime.strptime( a_row['post_time'], "%Y/%m/%d %H:%M:%S") 
        adj_time = time+time_delta
        adj_date = (adj_time + timedelta(days = 2)).date()
        title = a_row['title']
        content = a_row['content']
        title_plus_content = title + '＜' + content
        
        if int(adj_time.hour) in  range(0, 20) and adj_date in market_date_list: 
            try:
                news_content_plus_title[adj_date] += title_plus_content
                number_of_document[adj_date] += 1
            except:
                news_content_plus_title[adj_date] = title_plus_content
                number_of_document[adj_date] = 1

In [90]:
for article, dates in zip(  news_content_plus_title.values(), news_content_plus_title.keys()   ): 
    print(dates, len(article), number_of_document[dates])

2016-01-11 5668 10
2016-01-12 41691 56
2016-01-13 57869 94
2016-01-14 57469 94
2016-01-15 41911 64
2016-01-18 22563 29
2016-01-19 35569 57
2016-01-20 58604 91
2016-01-21 67359 104
2016-01-22 66895 102
2016-01-25 13145 20
2016-01-26 32239 49
2016-01-27 60549 103
2016-01-28 65087 106
2016-01-29 75827 119
2016-01-30 67858 103
2016-02-01 28800 43
2016-02-02 30694 42
2016-02-03 57649 90
2016-02-15 1327 5
2016-02-16 48063 63
2016-02-17 50188 75
2016-02-18 66136 85
2016-02-19 49960 80
2016-02-22 6199 12
2016-02-23 40822 52
2016-02-24 77859 109
2016-02-25 63761 92
2016-02-26 70503 108
2016-03-01 5350 9
2016-03-02 39010 72
2016-03-03 61664 98
2016-03-04 68522 110
2016-03-07 9159 14
2016-03-08 39910 49
2016-03-09 59060 90
2016-03-10 52697 84
2016-03-11 53050 86
2016-03-14 9988 15
2016-03-15 34648 47
2016-03-16 61209 93
2016-03-17 57177 91
2016-03-18 60443 100
2016-03-21 11553 18
2016-03-22 42631 48
2016-03-23 55817 80
2016-03-24 60453 96
2016-03-25 57892 94
2016-03-28 11016 16
2016-03-29 37688 4

In [115]:
vector_dict = dict()
for article, dates in zip(  news_content_plus_title.values(), news_content_plus_title.keys()   ):
    vector_list = []
    for key_word in keyword_list:
        vector_list.append( article.count(key_word) / number_of_document[dates])
    #標示target類別
    vector_list.append( target[dates] )
    vector_dict[dates] = vector_list

In [116]:
df_news = pd.DataFrame.from_dict(vector_dict, 'index', columns = keyword_list +[u'預測類別'])
df_news

,產金分離,台股開紅盤,參加富邦證券,股開紅盤,華為事件,安能得,參加日盛證券,金分離,華為財務長,波索納洛,...,櫃個股,單位千元,耶誕,增減百分比,月營收資料,敦北,關聯企業及合,聯企業及合資,定價日,預測類別
2016-01-11,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
2016-01-12,0.000000,0.017857,0.000000,0.017857,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
2016-01-13,0.000000,0.010638,0.000000,0.010638,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.010638,0.000000,0.000000,0.000000,1
2016-01-14,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
2016-01-15,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.015625,0.000000,0.000000,0.000000,0
2016-01-18,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1
2016-01-19,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1
2016-01-20,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.032967,0.000000,0.000000,0.000000,0
2016-01-21,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.009615,0.000000,0.000000,0.000000,0
2016-01-22,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1


In [117]:
df_news.describe()

,產金分離,台股開紅盤,參加富邦證券,股開紅盤,華為事件,安能得,參加日盛證券,金分離,華為財務長,波索納洛,...,櫃個股,單位千元,耶誕,增減百分比,月營收資料,敦北,關聯企業及合,聯企業及合資,定價日,預測類別
count,722.000000,722.000000,722.000000,722.000000,722.000000,722.000000,722.000000,722.000000,722.000000,722.000000,...,722.000000,722.000000,722.000000,722.000000,722.000000,722.000000,722.000000,722.000000,722.000000,722.000000
mean,0.000363,0.002679,0.000263,0.002810,0.000572,0.000564,0.000473,0.000744,0.000599,0.000515,...,0.000596,0.003764,0.007866,0.003744,0.003811,0.003617,0.002689,0.002689,0.000884,0.542936
std,0.004388,0.023731,0.002305,0.024219,0.006964,0.004769,0.006123,0.006206,0.006635,0.006842,...,0.001880,0.023610,0.050059,0.023664,0.023723,0.010285,0.025379,0.025379,0.004531,0.498498
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.001284,0.000000,0.000000,0.000000,1.000000
max,0.090652,0.400000,0.040506,0.400000,0.163636,0.075691,0.145833,0.106667,0.135135,0.152542,...,0.013514,0.338747,0.857143,0.338747,0.338747,0.142857,0.345643,0.345643,0.049587,1.000000


# 初步使用random forest來選取特徵值

In [119]:
import numpy as np

        
y = df_news[u'預測類別']  # we only take the first two features.
X = df_news.iloc[:, :-1]
#split
from sklearn.model_selection import train_test_split as tt_split
x_train,x_test,y_train,y_test = tt_split(X,y,test_size=0.3,random_state=0)

from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(criterion = 'entropy', n_estimators =100, random_state = 1, n_jobs = 2)


#fit
forest.fit(x_train,y_train)


from sklearn.model_selection import cross_val_score 

scores = cross_val_score(forest, X, y, cv=5, scoring='accuracy')
print(scores)
print(scores.mean())

print(forest.score(x_test, y_test))




[0.55862069 0.56551724 0.55555556 0.49305556 0.55555556]
0.5456609195402299
0.5714285714285714


In [120]:
y_pred = forest.predict(x_test)
y_pred

array([1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0,
       1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1,
       0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1,
       1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0,
       0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1],
      dtype=int64)

In [122]:
from sklearn.metrics import classification_report
y_pred = list(y_pred)
y_true = list(y_test)
target_names = ['class 0', 'class 1']
print(classification_report(y_true, y_pred, target_names=target_names))


              precision    recall  f1-score   support

     class 0       0.58      0.41      0.48       105
     class 1       0.57      0.72      0.64       112

   micro avg       0.57      0.57      0.57       217
   macro avg       0.57      0.57      0.56       217
weighted avg       0.57      0.57      0.56       217



## 可看出大部分有決定力的單字都是特定事件，而非常態出現的字

In [129]:
importances = forest.feature_importances_
indices = np.argsort(importances)[::-1]
for f in range(0,100):
    print('%2d) %-*s %f' %(f+1, 30, keyword_list[f], importances[indices][f]))

 1) 產金分離                           0.018419
 2) 台股開紅盤                          0.016008
 3) 參加富邦證券                         0.013493
 4) 股開紅盤                           0.013146
 5) 華為事件                           0.011604
 6) 安能得                            0.009922
 7) 參加日盛證券                         0.009344
 8) 金分離                            0.008960
 9) 華為財務長                          0.008864
10) 波索納洛                           0.008817
11) 保釋                             0.008670
12) 台北君悅酒店                         0.008345
13) 為事                             0.008218
14) 孟晚舟                            0.007772
15) 瑪莉亞                            0.007697
16) 蔡明忠                            0.007537
17) 禁售                             0.007519
18) 買回股份期間                         0.007516
19) 際已買回股數                         0.007457
20) 實際已買回股                         0.007424
21) 買回股數占預                         0.007289
22) 股數占預定買                         0.007276
23) 數占預定買回                      

## 配合人工選，我整理出新的特徵List

In [5]:
import pandas as pd
df = pd.read_csv('key2.csv', encoding = 'big5').dropna()
df

,word
0,倒掛
1,進場護盤
2,崩
3,是否進場
4,摜破
5,暫時休兵
6,一片慘綠
7,跳空開低
8,美股暴跌
9,貨幣危機


In [6]:
keyword_list2 = df['word'].tolist()
vector_dict = dict()
for article, dates in zip(  news_content_plus_title.values(), news_content_plus_title.keys()   ):
    vector_list = []
    for key_word in keyword_list2:
        vector_list.append( article.count(key_word) / number_of_document[dates])
    #標示target類別
    vector_list.append( target[dates] )
    vector_dict[dates] = vector_list

In [7]:
data = pd.DataFrame.from_dict(vector_dict, 'index', columns = keyword_list2 +[u'預測類別'])
data

,倒掛,進場護盤,崩,是否進場,摜破,暫時休兵,一片慘綠,跳空開低,美股暴跌,貨幣危機,...,目標收益,慘淡,量縮,大量,虧損,營收上升,營收下滑,獲利上升,獲利下滑,預測類別
2016-01-11,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
2016-01-12,0.000000,0.035714,0.053571,0.000000,0.000000,0.000000,0.000000,0.035714,0.000000,0.000000,...,0.000000,0.000000,0.107143,0.053571,0.035714,0.000000,0.000000,0.000000,0.000000,0
2016-01-13,0.000000,0.063830,0.042553,0.000000,0.021277,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.042553,0.042553,0.010638,0.000000,0.000000,0.000000,0.000000,1
2016-01-14,0.000000,0.000000,0.010638,0.000000,0.000000,0.000000,0.010638,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.053191,0.010638,0.000000,0.000000,0.000000,0.000000,0
2016-01-15,0.000000,0.000000,0.015625,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.093750,0.015625,0.125000,0.000000,0.000000,0.000000,0.000000,0
2016-01-18,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.103448,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1
2016-01-19,0.000000,0.035088,0.017544,0.017544,0.035088,0.000000,0.000000,0.017544,0.000000,0.000000,...,0.000000,0.017544,0.000000,0.052632,0.017544,0.000000,0.000000,0.000000,0.000000,1
2016-01-20,0.000000,0.032967,0.054945,0.000000,0.021978,0.000000,0.021978,0.043956,0.000000,0.000000,...,0.000000,0.010989,0.065934,0.021978,0.032967,0.000000,0.000000,0.000000,0.000000,0
2016-01-21,0.000000,0.000000,0.038462,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.057692,0.028846,0.048077,0.000000,0.019231,0.000000,0.009615,0
2016-01-22,0.000000,0.009804,0.117647,0.000000,0.039216,0.000000,0.009804,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.019608,0.068627,0.000000,0.000000,0.000000,0.000000,1


## 再從新train一次模型

## RF

In [23]:
import numpy as np
        
y = data[u'預測類別']  # we only take the first two features.
X = data.iloc[:, :-1]
#split
from sklearn.model_selection import train_test_split as tt_split
x_train,x_test,y_train,y_test = tt_split(X,y,test_size=0.3,random_state=0)

from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(criterion = 'entropy', n_estimators =100, random_state = 1, n_jobs = 2)

#fit
forest.fit(x_train,y_train)

# scores
from sklearn.model_selection import cross_val_score 
scores = cross_val_score(forest, X, y, cv=5, scoring='accuracy')
print(scores)
print(scores.mean())
print(forest.score(x_test, y_test))

#y_pred
y_pred = forest.predict(x_test)

[0.50344828 0.55172414 0.5625     0.55555556 0.47222222]
0.5290900383141762
0.5806451612903226


In [24]:
from sklearn.metrics import classification_report
y_pred = list(y_pred)
y_true = list(y_test)
target_names = ['class 0', 'class 1']
print(classification_report(y_true, y_pred, target_names=target_names))

              precision    recall  f1-score   support

     class 0       0.60      0.39      0.47       105
     class 1       0.57      0.76      0.65       112

   micro avg       0.58      0.58      0.58       217
   macro avg       0.59      0.57      0.56       217
weighted avg       0.59      0.58      0.57       217



## Logit

In [19]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(C=1000.0, random_state = 0)
lr.fit(x_train,y_train)

# scores
from sklearn.model_selection import cross_val_score 
scores = cross_val_score(lr, X, y, cv=5, scoring='accuracy')
print(scores)
print(scores.mean())
print(lr.score(x_test, y_test))

#y_pred
y_pred = lr.predict(x_test)

[0.53793103 0.55172414 0.52777778 0.5        0.52083333]
0.5276532567049809
0.5714285714285714


c:\users\user\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\user\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\user\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\user\appdata\local\programs\python\python37\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
c:\users\user\appdata\local\programs\python\python37\lib\site-pa

In [20]:
y_pred = list(y_pred)
y_true = list(y_test)
target_names = ['class 0', 'class 1']
print(classification_report(y_true, y_pred, target_names=target_names))

              precision    recall  f1-score   support

     class 0       0.57      0.44      0.50       105
     class 1       0.57      0.70      0.63       112

   micro avg       0.57      0.57      0.57       217
   macro avg       0.57      0.57      0.56       217
weighted avg       0.57      0.57      0.56       217



## svm

In [21]:
from sklearn.svm import SVC
svm = SVC(kernel = 'linear', C=1.0, random_state = 0)
svm.fit(x_train,y_train)

# scores
from sklearn.model_selection import cross_val_score 
scores = cross_val_score(svm, X, y, cv=5, scoring='accuracy')
print(scores)
print(scores.mean())
print(svm.score(x_test, y_test))

#y_pred
y_pred = svm.predict(x_test)

[0.52413793 0.54482759 0.54861111 0.53472222 0.55555556]
0.5415708812260537
0.5207373271889401


In [22]:
y_pred = list(y_pred)
y_true = list(y_test)
target_names = ['class 0', 'class 1']
print(classification_report(y_true, y_pred, target_names=target_names))

              precision    recall  f1-score   support

     class 0       0.60      0.03      0.05       105
     class 1       0.52      0.98      0.68       112

   micro avg       0.52      0.52      0.52       217
   macro avg       0.56      0.51      0.37       217
weighted avg       0.56      0.52      0.38       217



## knn

In [26]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = 5, p=2)
knn.fit(x_train,y_train)

# scores
from sklearn.model_selection import cross_val_score 
scores = cross_val_score(knn, X, y, cv=5, scoring='accuracy')
print(scores)
print(scores.mean())
print(knn.score(x_test, y_test))

#y_pred
y_pred = knn.predict(x_test)

[0.51724138 0.55862069 0.49305556 0.54166667 0.54861111]
0.5318390804597701
0.4930875576036866


In [27]:
y_pred = list(y_pred)
y_true = list(y_test)
target_names = ['class 0', 'class 1']
print(classification_report(y_true, y_pred, target_names=target_names))

              precision    recall  f1-score   support

     class 0       0.47      0.43      0.45       105
     class 1       0.51      0.55      0.53       112

   micro avg       0.49      0.49      0.49       217
   macro avg       0.49      0.49      0.49       217
weighted avg       0.49      0.49      0.49       217

